In [1]:
import json
import re
from collections import defaultdict

# Sample text input (truncated for brevity)
text = """
NGHỊ ĐỊNH
QUY ĐỊNH QUẢN LÝ ĐẦU TƯ ỨNG DỤNG CÔNG NGHỆ THÔNG TIN SỬ DỤNG NGUỒN VỐN NGÂN SÁCH NHÀ NƯỚC
Căn cứ Luật tổ chức Chính phủ ngày 19 tháng 6 năm 2015;
...
Chương I
NHỮNG QUY ĐỊNH CHUNG
Điều 1. Phạm vi điều chỉnh
1. Nghị định này quy định quản lý các hoạt động ứng dụng công nghệ thông tin sau đây:
a) Dự án ứng dụng công nghệ thông tin sử dụng kinh phí chi đầu tư phát triển nguồn vốn ngân sách nhà nước;
b) Các hoạt động ứng dụng công nghệ thông tin sử dụng kinh phí chi thường xuyên nguồn vốn ngân sách nhà nước.
2. Đối với dự án ứng dụng công nghệ thông tin sử dụng hỗn hợp nhiều nguồn vốn khác nhau...
Điều 2. Đối tượng áp dụng
1. Nghị định này áp dụng đối với cơ quan, tổ chức...
Điều 3. Giải thích từ ngữ
1. Báo cáo kinh tế-kỹ thuật dự án đầu tư ứng dụng công nghệ thông tin...
...
"""

# Define regex patterns for each level
CHUONG_PATTERN = re.compile(r"^Chương\s+([IVXLCDM]+|\d+)", re.IGNORECASE)
DIEU_PATTERN = re.compile(r"^Điều\s+(\d+[a-zđ]?)", re.IGNORECASE)
KHOAN_PATTERN = re.compile(r"^(\d+)\.\s*(.*)")
MUC_PATTERN = re.compile(r"^([a-zđA-ZĐ])\)\s*(.*)")


def parse_text(text):
    hierarchy = []
    current_chuong = None
    current_dieu = None
    current_khoan = None
    current_muc = None

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue  # Skip empty lines

        # Check for Chương
        chuong_match = CHUONG_PATTERN.match(line)
        if chuong_match:
            chuong_title = line
            current_chuong = {"Chuong": chuong_title, "Dieu": []}
            hierarchy.append(current_chuong)
            current_dieu = None
            current_khoan = None
            current_muc = None
            continue

        # Check for Điều
        dieu_match = DIEU_PATTERN.match(line)
        if dieu_match:
            dieu_number = dieu_match.group(1)
            dieu_content = line[dieu_match.end() :].strip(". ").strip()
            current_dieu = {
                "Dieu": f"Dieu {dieu_number}",
                "Content": dieu_content,
                "Khoan": [],
            }
            if current_chuong is None:
                # If no Chương, create a default one
                current_chuong = {"Chuong": "Chuong 0", "Dieu": []}
                hierarchy.append(current_chuong)
            current_chuong["Dieu"].append(current_dieu)
            current_khoan = None
            current_muc = None
            continue

        # Check for Khoản
        khoan_match = KHOAN_PATTERN.match(line)
        if khoan_match and current_dieu is not None:
            khoan_number = khoan_match.group(1)
            khoan_content = khoan_match.group(2)
            current_khoan = {
                "Khoan": f"Khoan {khoan_number}",
                "Content": khoan_content,
                "Muc": [],
            }
            current_dieu["Khoan"].append(current_khoan)
            current_muc = None
            continue

        # Check for Mục
        muc_match = MUC_PATTERN.match(line)
        if muc_match and current_khoan is not None:
            muc_letter = muc_match.group(1)
            muc_content = muc_match.group(2)
            current_muc = {
                "Muc": f"Muc {muc_letter}",
                "Content": muc_content,
            }
            current_khoan["Muc"].append(current_muc)
            continue

        # If none of the above, it's part of the current content
        if current_muc:
            current_muc["Content"] += " " + line
        elif current_khoan:
            current_khoan["Content"] += " " + line
        elif current_dieu:
            current_dieu["Content"] += " " + line
        elif current_chuong:
            current_chuong["Content"] = (
                current_chuong.get("Content", "") + " " + line
            )
        else:
            # Content before any Chương or Điều
            if "Preamble" not in hierarchy:
                hierarchy.append({"Preamble": line})
            else:
                hierarchy[-1]["Preamble"] += " " + line

    return hierarchy

In [3]:
import pandas as pd

# Load data from a CSV file
data = pd.read_csv("luat.csv")

# Display the first few rows of the dataframe
data = data[
    data["subject"].apply(lambda x: "mới nhất" in x and "sửa đổi" not in x)
]
hierarchical_structures = {}
for subject, text in data[["subject", "text"]].values:
    hierarchical_structures[subject] = parse_text(text)


def get_text_from_structure(dieu, kdm=False):
    head = dieu["Dieu"].replace("Dieu", "Điều")
    head = f"{head}. {dieu['Content']}\n"
    content = f"{head}"
    clause_annotation = ""
    item_annotation = ""
    clause_prefix = ""
    item_prefix = ""
    if kdm:
        clause_annotation = "Khoản "
        item_annotation = "Điểm "
        clause_prefix = dieu["Dieu"].replace("Dieu", "Điều") + " "
    for khoan in dieu["Khoan"]:
        content += f"{khoan['Khoan'].replace('Khoan ',clause_annotation)}. {clause_prefix}{khoan['Content']}\n"
        if kdm:
            item_prefix = f"{khoan['Khoan'].replace('Khoan ',clause_annotation)} {clause_prefix} "
        if len(khoan["Muc"]) > 0:
            for muc in khoan["Muc"]:
                content += f"{muc['Muc'].replace('Muc ',item_annotation)}) {item_prefix}{muc['Content']}\n"
    return content

In [4]:
articles = []
documents = []
for law in hierarchical_structures:
    for element in hierarchical_structures[law]:
        if "Chuong" in element:
            # chuong_header = f"Chương {element['Chuong']}: {element['Content']}\n"
            # content = chuong_header
            citation = law
            for dieu in element["Dieu"]:
                article_name = dieu["Dieu"].replace("Dieu", "Điều")
                content = get_text_from_structure(dieu, kdm=False)
                if re.search("Mục [0-9]+[a-zđ]?. ", content):
                    # print(content)
                    content = re.sub("Mục [0-9]+[a-zđ]?. .+?$", "", content)
                if re.search("Luật này (đã )?được Quốc hội", content):
                    content = re.sub(
                        "Luật này (đã )?được Quốc hội.+?$",
                        "",
                        content,
                        flags=re.DOTALL,
                    )
                doc = {
                    "title": article_name,
                    "content": content,
                    "citation": article_name + " " + citation,
                    "law": law,
                }

                articles.append(content)
                documents.append(doc)

In [5]:
from tqdm import tqdm

for doc in tqdm(documents):
    print(doc)
    break

  0%|                                                 | 0/19010 [00:00<?, ?it/s]

{'title': 'Điều 1', 'content': 'Điều 1. Nhiệm vụ của Bộ luật hình sự Bộ luật hình sự có nhiệm vụ bảo vệ chủ quyền quốc gia, an ninh của đất nước, bảo vệ chế độ xã hội chủ nghĩa, quyền con người, quyền công dân, bảo vệ quyền bình đẳng giữa đồng bào các dân tộc, bảo vệ lợi ích của Nhà nước, tổ chức, bảo vệ trật tự pháp luật, chống mọi hành vi phạm tội; giáo dục mọi người ý thức tuân theo pháp luật, phòng ngừa và đấu tranh chống tội phạm. Bộ luật này quy định về tội phạm và hình phạt.\n', 'citation': 'Điều 1 Bộ luật hình sự 2015 số 100/2015/QH13 mới nhất', 'law': 'Bộ luật hình sự 2015 số 100/2015/QH13 mới nhất'}


In [6]:
import chromadb

client = chromadb.PersistentClient(path="./chromadb")
collection = client.get_collection(name="luat_vn")

In [7]:
from llama_index.core import Document
from llama_index.core.schema import (
    NodeRelationship,
    RelatedNodeInfo,
    TextNode,
)

nodes = []
for c in documents:
    citation = c["citation"]
    article_node = TextNode(
        text=c["content"],
        metadata={
            "name": c["title"],
            "type": "Điều",
            "citation": citation,
            "law": c["law"],
        },
    )
    nodes.append(article_node)

In [8]:
# llamaindex nodes with title, title is the first element of chunks, content is the second
import torch
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb

# index nodes using vectorstore


# Importing ServiceContext is removed due to its deprecation
client = chromadb.PersistentClient(path="./chromadb_all_f")
collection = client.get_or_create_collection(name="all_vn_laws")
# Initialize embedding model
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
model_name = "/media/halean/3TB_1/cache/hub/models--Alibaba-NLP--gte-Qwen2-1.5B-instruct/snapshots/c6c1b92f4a3e1b92b326ad29dd3c8433457df8dd/"
model_kwargs = {
    "trust_remote_code": True,
    # "torch_dtype": torch.float16,
    "device": "cuda",
    # "model_kwargs":{"torch_dtype": torch.float16}
}  # , "torch_dtype": torch.float16}#, "model_kwargs":{"torch_dtype": torch.float16}}
embed_model = HuggingFaceEmbedding(model_name=model_name, **model_kwargs)

# Initialize vector store (ensure chroma_collection is properly defined)
vector_store = ChromaVectorStore(chroma_collection=collection)
# Initialize storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)
from llama_index.core.schema import TextNode

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    show_progress=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/578 [00:00<?, ?it/s]

In [9]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)


async def get_related_regulations(query):
    nodes = await retriever.aretrieve(query)
    return nodes

In [10]:
f = """
Trong thời gian lãnh đạo Phòng Cảnh sát giao thông, ông Tiến bị cáo buộc không thực hiện đúng chức năng, nhiệm vụ được giao; không kiểm tra, đôn đốc Lê Hữu Nghĩa, Đội phó Tổng hợp - phụ trách kế toán, thực hiện việc nộp tiền lệ phí cấp giấy đăng ký và biển số phương tiện giao thông cơ giới đường bộ về Phòng Hậu cần để nộp vào kho bạc và thực hiện khai thuế theo quy định của pháp luật.

Nhà chức trách cáo buộc, sự thiếu trách nhiệm của ông Tiến dẫn đến việc Nghĩa giữ lại 280 cuốn biên lai thu lệ phí đăng ký đã sử dụng và chiếm đoạt hơn 3,5 tỷ đồng tiền thu lệ phí trong thời gian từ tháng 1/2016 đến tháng 11/2019 để sử dụng mục đích cá nhân.

Hậu quả thiệt hại cho hành vi thiếu trách nhiệm của ông Tiến gây ra là hơn 3,5 tỷ đồng tiền thu lệ phí cấp giấy đăng ký và biển số phương tiện giao thông cơ giới đường bộ, chưa được nộp vào ngân sách Nhà nước.
"""
f = """
khiếu nại hàng xóm trổ cửa ra hẻm
"""
nodes_re = await get_related_regulations(f)
for node in nodes_re:
    print(f"{node.text}\n{node.metadata['law']}\n--------\n")

Điều 178. Trổ cửa nhìn sang bất động sản liền kề
1. Chủ sở hữu nhà chỉ được trổ cửa ra vào, cửa sổ quay sang nhà bên cạnh, nhà đối diện và đường đi chung theo quy định của pháp luật về xây dựng.
2. Mặt dưới mái che trên cửa ra vào, mặt dưới mái che cửa sổ quay ra đường đi chung phải cách mặt đất từ 2,5 mét trở lên.

Bộ luật dân sự 2015 số 91/2015/QH13 mới nhất
--------

Điều 16. 
1. Việc mở cửa khẩu và nơi mở ra cho qua lại biên giới, nâng cấp cửa khẩu, đóng cửa khẩu, xác định các tuyến đường bộ, đường sắt, đường thuỷ nội địa, đường hàng hải, đường hàng không dùng cho việc quá cảnh do Chính phủ quyết định theo quy định của pháp luật Việt Nam hoặc điều ước quốc tế mà Việt Nam ký kết hoặc gia nhập.
2. Việc ra, vào cửa khẩu, tạm trú và các hoạt động khác ở khu vực cửa khẩu phải tuân theo quy định của pháp luật.

Luật Biên giới Quốc gia 2003 số 06/2003/QH11 mới nhất
--------

Điều 18. Cấp thị thực tại cửa khẩu quốc tế
1. Người nước ngoài được cấp thị thực tại cửa khẩu quốc tế trong các trư

In [16]:
all_laws = """
Bộ luật hình sự 2015 số 100/2015/QH13 mới nhất
Luật thuế xuất khẩu thuế nhập khẩu 2016 số 107/2016/QH13 mới nhất
Luật tài nguyên môi trường biển và hải đảo 2015 số 82/2015/QH13 mới nhất
Luật Dược 2016 số 105/2016/QH13 mới nhất
Luật Đầu tư công 2019 số 39/2019/QH14 mới nhất
Luật năng lượng nguyên tử 2008 số 18/2008/QH12 mới nhất
Luật giáo dục 2019 số 43/2019/QH14 mới nhất áp dụng 2024 mới nhất
Luật thi hành án dân sự 2008 số 26/2008/QH12 mới nhất
Luật đo lường 2011 số 04/2011/QH13 mới nhất
Luật Điện ảnh 2022 mới nhất
Luật thuế sử dụng đất phi nông nghiệp 2010 số 48/2010/QH12 mới nhất
Luật Căn cước năm 2023 số 26/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật giá năm 2012 số 11/2012/QH13 mới nhất
Luật Xây dựng 2014 số 50/2014/QH13 mới nhất
Luật thuế bảo vệ môi trường 2010 số 57/2010/QH12 mới nhất
Luật Viễn thông 2023 số 24/2023/QH15 mới nhất áp dụng 2024 mới nhất
"Luật phòng, chống nhiễm vi rút gây ra hội chứng suy giảm miễn dịch mắc phải ở người HIV/AIDS 2006 64/2006/QH11 mới nhất"
Bộ luật dân sự 2015 số 91/2015/QH13 mới nhất
"Luật phòng, chống tác hại của thuốc lá 2012 số 09/2012/QH13 mới nhất"
Luật Nhập cảnh xuất cảnh quá cảnh cư trú của người nước ngoài tại Việt Nam 2014 số 47/2014/QH13 mới nhất
Luật Giáo dục nghề nghiệp 2014 số 74/2014/QH13 mới nhất
Luật phòng cháy chữa cháy 2001 số 27/2001/QH10 mới nhất
Luật hợp tác xã 2012-23/2012/QH13 mới nhất
"Luật trưng mua, trưng dụng tài sản 2008 số 15/2008/QH12 mới nhất"
Luật viên chức 2010 số 58/2010/QH12 mới nhất
Luật trẻ em 2016 số 102/2016/QH13 mới nhất
Luật quốc tịch Việt Nam 2008 số 24/2008/QH12 mới nhất
Luật quản lý bảo vệ công trình quốc phòng và khu quân sự năm 2023 số 25/2023/QH15 mới nhất mới nhất
Luật quyền lập hội 1957 102-SL/L-004 mới nhất
Luật Biên giới Quốc gia 2003 số 06/2003/QH11 mới nhất
Luật Phòng chống tác hại của rượu bia 2019 số 44/2019/QH14 mới nhất
Luật an toàn thực phẩm 2010 số 55/2010/QH12 mới nhất
Luật công nghệ cao 2008 số 21/2008/QH12 mới nhất
Luật Giao thông đường thủy nội địa 2004 số 23/2004/QH11 mới nhất
Luật Trọng tài thương mại 2010 số 54/2010/QH12 mới nhất
Luật bưu chính 2010 số 49/2010/QH12 mới nhất
Luật Người lao động Việt Nam đi làm việc ở nước ngoài theo hợp đồng 2020 số 69/2020/QH14 mới nhất
Luật Bình đẳng giới 2006 số 73/2006/QH11 mới nhất
Luật Giáo dục quốc phòng và an ninh năm 2013 số 30/2013/QH13 mới nhất
Luật đấu giá tài sản 2016 số 01/2016/QH14 mới nhất
Luật tín ngưỡng tôn giáo 2016 số 02/2016/QH14 mới nhất
Luật thuế tiêu thụ đặc biệt 2008 số 26/2008/QH12 mới nhất
Luật Kinh doanh bất động sản 2014 số 66/2014/QH13 mới nhất
Luật bảo vệ quyền lợi người tiêu dùng 2010 số 59/2010/QH12 mới nhất
Luật sử dụng năng lượng tiết kiệm và hiệu quả 2010 số 50/2010/QH12 mới nhất
Luật Cảnh vệ 2017 số 13/2017/QH14 mới nhất
Luật Lực lượng dự bị động viên 2019 số 53/2019/QH14 mới nhất
Luật Chăn nuôi 2018 số 32/2018/QH14 mới nhất
Luật tiếp cận thông tin 2016 số 104/2016/QH13 mới nhất
Luật Cảnh sát cơ động 2022 mới nhất
Luật viễn thông năm 2009 số 41/2009/QH12 mới nhất
Luật Thủ đô 2012 -25/2012/QH13 mới nhất
Luật lưu trữ 2011 số 01/2011/QH13 mới nhất
Luật Phòng chống rửa tiền 2022 số 14/2022/QH15 mới nhất áp dụng 2024 mới nhất
Luật Phòng chống ma túy 2021 số 73/2021/QH14 mới nhất
Luật Nhà ở 2023 số 27/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật tài nguyên nước 2012 số 17/2012/QH13 mới nhất
Luật Thực hiện dân chủ ở cơ sở năm 2022 mới nhất
Luật Bảo vệ sức khỏe nhân dân 1989 số 21-LCT/HĐNN8 mới nhất
Luật đê điều 2006 79/2006/QH11 mới nhất
Luật di sản văn hoá 2001 số 28/2001/QH10 mới nhất
Luật Đất đai 2024 số 31/2024/QH15  mới nhất
Luật Bảo hiểm xã hội 2014 số 58/2014/QH13 mới nhất
Luật Hôn nhân và gia đình 2014 số 52/2014/QH13 mới nhất
Luật Tố cáo 2018 số 25/2018/QH14 mới nhất
Luật an toàn thông tin mạng 2015 số 86/2015/QH13 mới nhất
Luật Thương mại 2005 số 36/2005/QH11 mới nhất
Luật Tiêu chuẩn và quy chuẩn kỹ thuật 2006 68/2006/QH11 mới nhất
Luật tổ chức Tòa án nhân dân 2014 số 62/2014/QH13 mới nhất
Luật Lực lượng tham gia bảo vệ an ninh trật tự ở cơ sở năm 2023 số 30/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Thi hành án hình sự 2019 số 41/2019/QH14 mới nhất
Luật kiểm toán nhà nước 2015 số 81/2015/QH13 mới nhất
Luật giám định tư pháp 2012 số 13/2012/QH13 mới nhất
Luật người khuyết tật 2010 số 51/2010/QH12 mới nhất
Luật hoạt động chữ thập đỏ 2008 số 11/2008/QH12 mới nhất
Luật quốc phòng năm 2018 số 22/2018/QH14 mới nhất
Luật quản lý thuế 2019 số 38/2019/QH14 mới nhất
Luật tố tụng hành chính 2015 số 93/2015/QH13 mới nhất
Bộ luật hàng hải Việt Nam 2015 số 95/2015/QH13 mới nhất
Luật Bảo hiểm tiền gửi 2012 số 06/2012/QH13 mới nhất
Luật thi hành tạm giữ tạm giam 2015 số 94/2015/QH13 mới nhất
Luật các tổ chức tín dụng 2010 số 47/2010/QH12 mới nhất
Luật tiếp công dân 2013 số 42/2013/QH13 mới nhất
Luật Giao dịch điện tử 2023 số 20/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Đường sắt 2017 số 06/2017/QH14 mới nhất
Luật Thư viện 2019 số 46/2019/QH14 mới nhất
Luật Hải quan 2014 số 54/2014/QH13 mới nhất
Luật khí tượng thủy văn 2015 số 90/2015/QH13 mới nhất
Luật Phòng chống tham nhũng năm 2018 số 36/2018/QH14 mới nhất
Luật phí và lệ phí 2015 số 97/2015/QH13 mới nhất
Bộ luật lao động 2019 số 45/2019/QH14 mới nhất áp dụng 2024 mới nhất
Luật An ninh Quốc gia 2004 số 32/2004/QH11 mới nhất
Luật biển Việt Nam 2012 số 18/2012/QH13 mới nhất
Luật phòng chống thiên tai năm 2013 số 33/2013/QH13 mới nhất
Luật Đầu tư 2020 số 61/2020/QH14 áp dụng năm 2024 mới nhất
Luật Cơ quan đại diện nước Cộng hòa xã hội chủ nghĩa Việt Nam tại nước ngoài năm 2017 số 19/2017/QH14 mới nhất
Luật Quản lý sử dụng tài sản công 2017 số 15/2017/QH14 mới nhất
Luật tần số vô tuyến điện năm 2009 số 42/2009/QH12 mới nhất
Luật mẫu về trọng tài thương mại quốc tế của Ủy ban liên hiệp quốc về Luật thương mại quốc tế 1985 mới nhất
Luật thuế Sử dụng Đất Nông nghiệp 1993 23-L/CTN mới nhất
Luật Hòa giải đối thoại tại Tòa án số 58/2020/QH14 mới nhất
Luật Hộ tịch 2014 số 60/2014/QH13 mới nhất
Luật Đặc xá 2018 số 30/2018/QH14 mới nhất
"Luật chất lượng sản phẩm, hàng hóa 2007 số 05/2007/QH12 mới nhất"
Luật hòa giải ở cơ sở năm 2013 số 35/2013/QH13 mới nhất
Luật phổ biến giáo dục pháp luật 2012 số 14/2012/QH13 mới nhất
Luật bảo vệ kiểm dịch thực vật 2013 số 41/2013/QH13 mới nhất
Luật Nhà ở 2014 số 65/2014/QH13 mới nhất
Luật điều ước quốc tế 2016 số 108/2016/QH13 mới nhất
Luật Quản lý nợ công năm 2017 số 20/2017/QH14 mới nhất
Luật hóa chất 2007 số 06/2007/QH12 mới nhất
Luật Báo chí 2016 số 103/2016/QH13 mới nhất
Luật giao thông đường bộ 2008 23/2008/QH12 mới nhất
Luật xuất bản 2012 số 19/2012/QH13 mới nhất
Luật chuyển giao công nghệ 2017 số 07/2017/QH14 mới nhất
Luật xử lý vi phạm hành chính 2012 số 15/2012/QH13 mới nhất
Luật Cảnh sát biển Việt Nam 2018 số 33/2018/QH14 mới nhất
Luật kiểm toán độc lập 2011 số 67/2011/QH12  mới nhất
Luật Công an nhân dân năm 2018 số 37/2018/QH14 mới nhất
Luật Hỗ trợ doanh nghiệp nhỏ và vừa 2017 số 04/2017/QH14 mới nhất
Luật Kiến trúc 2019 số 40/2019/QH14 mới nhất
Luật Dân quân tự vệ 2019 số 48/2019/QH14 mới nhất
Luật người cao tuổi năm 2009 số 39/2009/QH12 mới nhất
Luật Trách nhiệm bồi thường của Nhà nước 2017 số 10/2017/QH14 mới nhất
Luật Biên phòng Việt Nam 2020 số38/2015/QH14 mới nhất
Luật Công chứng 2014 số 53/2014/QH13 mới nhất
Luật Sĩ quan Quân đội nhân dân Việt Nam 1999 năm 16/1999/QH10 mới nhất
Luật Giá 2023 số 16/2023/QH15 mới nhất áp dụng 2024 mới nhất
"Luật Thể dục, Thể thao 2006 số 77/2006/QH11 mới nhất"
Luật quy hoạch 2017 số 21/2017/QH14 mới nhất
Luật Xuất cảnh nhập cảnh của công dân Việt Nam 2019 số 49/2019/QH14 mới nhất
Luật nuôi con nuôi 2010 số 52/2010/QH12 mới nhất
Luật an toàn vệ sinh lao động 2015 số 84/2015/QH13 mới nhất
Luật trưng cầu ý dân 2015 số 96/2015/QH13 mới nhất
Luật Điện Lực 2004 số 28/2004/QH11 mới nhất
Luật Quản lý ngoại thương 2017 số 05/2017/QH14 mới nhất
Luật khiếu nại 2011 số 02/2011/QH13 mới nhất
Luật Doanh nghiệp 2020 số 59/2020/QH14 áp dụng năm 2024 mới nhất
Luật quy hoạch đô thị 2009 30/2009/QH12 mới nhất
Luật Dầu khí 2022 số 12/2022/QH15  mới nhất áp dụng 2024 mới nhất
Luật quân nhân chuyên nghiệp công nhân viên chức quốc phòng 2015 số 98/2015/QH13 mới nhất
Luật dự trữ quốc gia 2012 số 22/2012/QH13 mới nhất
Luật lý lịch tư pháp 2009 số 28/2009/QH12 mới nhất
Luật tương trợ tư pháp 2007 số 08/2007/QH12 mới nhất
Luật Quản lý sử dụng vũ khí vật liệu nổ công cụ hỗ trợ 2017 số 14/2017/QH14 mới nhất
Luật Các tổ chức tín dụng 2024 mới nhất số 32/2024/QH15  mới nhất
Luật lâm nghiệp 2017 số 16/2017/QH14 mới nhất
Luật Kinh doanh bất động sản năm 2023 số 29/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật đa dạng sinh học 2008 số 20/2008/QH12 mới nhất
Luật Phòng thủ dân sự 2023 số 18/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Trồng trọt 2018 số 31/2018/QH14 mới nhất
Luật Tổ chức Viện kiểm sát nhân dân 2014 số 63/2014/QH13 mới nhất
Luật Giáo dục đại học 2012 số 08/2012/QH13 mới nhất
Luật cạnh tranh năm 2018 số 23/2018/QH14 mới nhất
Luật hàng không dân dụng Việt Nam 2006 số 66/2006/QH11 mới nhất
Luật khoáng sản 2010 số 60/2010/QH12 mới nhất
Luật Công đoàn 2012 số 12/2012/QH13 mới nhất
Luật Bảo vệ bí mật nhà nước 2018 số 29/2018/QH14 mới nhất
Luật thuế giá trị gia tăng 2008 số 13/2008/QH12 mới nhất
Luật Tổ chức Quốc hội 2014 số 57/2014/QH13 mới nhất
Luật Tài nguyên nước 2023 số 28/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Giao dịch điện tử 2005 số 51/2005/QH11 mới nhất
Luật công nghệ thông tin 2006 số 67/2006/QH11 mới nhất
Luật Các công cụ chuyển nhượng 2005 số 49/2005/QH11 mới nhất
Bộ luật tố tụng hình sự 2015 số 101/2015/QH13 mới nhất
"Luật cán bộ, công chức 2008 số 22/2008/QH12 mới nhất"
Luật thuế tài nguyên năm 2009 số 45/2009/QH12 mới nhất
Luật Phòng chống bạo lực gia đình 2022 số 13/2022/QH15 mới nhất áp dụng 2024 mới nhất
Luật Phá sản 2014 số 51/2014/QH13 mới nhất
Bộ luật tố tụng dân sự 2015 số 92/2015/QH13 mới nhất
Luật Kinh doanh bảo hiểm 2022 mới nhất
Luật tổ chức chính quyền địa phương 2015 số 77/2015/QH13 mới nhất
Luật bảo hiểm y tế 2008 số 25/2008/QH12 mới nhất
Luật khám bệnh chữa bệnh 2023 số 15/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Bảo vệ môi trường 2020 số 72/2020/QH14 mới nhất
Luật Bầu cử đại biểu Quốc hội và đại biểu Hội đồng nhân dân 2015 số 85/2015/QH13 mới nhất
Luật khoa học và công nghệ năm 2013 số 29/2013/QH13  mới nhất
Luật ban hành văn bản quy phạm pháp luật 2015 số 80/2015/QH13 mới nhất
Luật thực hành tiết kiệm chống lãng phí 2013 số 44/2013/QH13 mới nhất
Luật Hợp tác xã 2023 số 17/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Thi đua Khen thưởng 2022 mới nhất
Luật Đấu thầu 2023 số 22/2023/QH15 mới nhất áp dụng năm 2024 mới nhất
Luật Quảng cáo 2012 số 16/2012/QH13 mới nhất
Luật Trợ giúp pháp lý 2017 số 11/2017/QH14 mới nhất
Luật cơ quan đại diện nước Cộng hòa xã hội chủ nghĩa Việt Nam ở nước ngoài 2009 33/2009/QH12 mới nhất
Luật Chứng khoán năm 2019 số 54/2019/QH14 mới nhất
Luật thuế thu nhập doanh nghiệp 2008 số 14/2008/QH12 mới nhất
Luật Cơ yếu 2011 số 05/2011/QH13 mới nhất
Luật thuế thu nhập cá nhân 2007 số 04/2007/QH12 mới nhất
Luật Thủy sản 2017 số 18/2017/QH14 mới nhất
Luật Đầu tư theo phương thức đối tác công tư 2020 số 64/2020/QH14 mới nhất
Luật Mặt trận tổ quốc Việt Nam 2015 số 75/2015/QH13 mới nhất
Luật nghĩa vụ quân sự 2015 số 78/2015/QH13 mới nhất
Luật Ngân hàng Nhà nước Việt Nam 2010 số 46/2010/QH12 mới nhất
Luật thú y 2015 số 79/2015/QH13 mới nhất
Luật Quản lý sử dụng vốn Nhà nước đầu tư vào sản xuất kinh doanh tại doanh nghiệp 2014 số 69/2014/QH13 mới nhất
Luật Sở hữu trí tuệ 2005 số 50/2005/QH11 mới nhất
Luật Bảo vệ quyền lợi người tiêu dùng 2023 số 19/2023/QH15 mới nhất áp dụng 2024 mới nhất
Luật Thanh tra 2022 số 11/2022/QH15 ban hành ngày 14/11/2022 mới nhất mới nhất
Luật Căn cước công dân 2014 số 59/2014/QH13 mới nhất
Luật quyền tự do hội họp 1957 101-SL/L.003 mới nhất
Luật tổ chức cơ quan điều tra hình sự 2015 số 99/2015/QH13 mới nhất
Luật du lịch 2017 số 09/2017/QH14 mới nhất
"Luật phòng, chống bệnh truyền nhiễm 2007 số 03/2007/QH12 mới nhất"
Luật an ninh mạng 2018 số 24/2018/QH14 mới nhất
Luật Thỏa thuận quốc tế 2020 số 70/2020/QH14 mới nhất
Luật hoạt động giám sát của Quốc hội và Hội đồng nhân dân 2015 số 87/2015/QH13 mới nhất
Luật đất đai 2013 số 45/2013/QH13 mới nhất
Luật phòng chống khủng bố năm 2013 số 28/2013/QH13 mới nhất
Luật Thủy lợi 2017 số 08/2017/QH14 mới nhất
"Luật phòng, chống mua bán người 2011 số 66/2011/QH12 mới nhất"
"Luật việc hiến, lấy, ghép mô, bộ phận cơ thể người và hiến lấy xác 2006 số 75/2006/QH11 mới nhất"
Luật việc làm năm 2013 số 38/2013/QH13 mới nhất
Luật kế toán 2015 số 88/2015/QH13 mới nhất
Luật luật sư 2006 số 65/2006/QH11 mới nhất
Luật Đo đạc và Bản đồ 2018 số 27/2018/QH14 mới nhất
Luật tổ chức Chính phủ 2015 số 76/2015/QH13 mới nhất
Luật thanh niên 2020 số 57/2020/QH14 mới nhất
Luật thống kê 2015 số 89/2015/QH13 mới nhất
Luật cư trú 2020 số 68/2020/QH14 áp dụng năm 2024 mới nhất
Luật ngân sách nhà nước năm 2015 số 83/2015/QH13 mới nhất
"""

prompt = f"""trong các luật sau, luật nào có liên quan tới việc {f}. Trả lời dưới dạng một json list. 
{all_laws}
"""
prompt
import railguards

selected_laws = await railguards.get_chat_response(prompt, "")
selected_laws =

Getting LLM response
Got LLM response
```json
[
    "Luật Đất đai 2024 số 31/2024/QH15  mới nhất",
    "Luật Xây dựng 2014 số 50/2014/QH13 mới nhất",
    "Bộ luật dân sự 2015 số 91/2015/QH13 mới nhất",
    "Luật khiếu nại 2011 số 02/2011/QH13 mới nhất"
]
```


In [64]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import (
    FilterOperator,
    MetadataFilter,
    MetadataFilters,
)

applicable_laws = json.loads(
    selected_laws.replace("```json", "").replace("```", "")
)
filter_func = lambda metadata: metadata.get("law") in applicable_laws

filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="law",
            operator=FilterOperator.IN,
            value=applicable_laws,  # all_laws.split("\n")
        )
    ]
)

In [69]:
retrieved_nodes = []
for applicable_law in applicable_laws:
    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="law",
                operator=FilterOperator.EQ,
                value=applicable_law,  # all_laws.split("\n")
            )
        ]
    )
    retriever = VectorIndexRetriever(
        index=index,
        filters=filters,
        # You can adjust the number of results retrieved
        similarity_top_k=2,
    )
    nodes_re = await get_related_regulations(f)
    retrieved_nodes.extend(nodes_re)

In [92]:
prompt = ""
for node in retrieved_nodes:
    
    prompt += (f"{node.metadata['citation']}\n{node.text}\n--------\n")
system_prompt = (
    "Bạn là một trợ lý ảo dành cho công dân viết đơn khiếu nại. Giúp tôi soạn đơn dựa vào các điều khoản có liên quan. "
    # "dùng mẫu đơn sau:"
    # f"{mau_don}"
    # "dùng mẫu đơn để 
    #" soạn đơn, "
    "nhớ trích dẫn các điều khoản quan trọng."
)
system_prompt
# system_prompt = " ".join(list(system_prompt))

'Bạn là một trợ lý ảo dành cho công dân viết đơn khiếu nại. Giúp tôi soạn đơn dựa vào các điều khoản có liên quan. nhớ trích dẫn các điều khoản quan trọng.'

In [93]:
async def get_llm_answer(qa, model="accounts/fireworks/models/llama-v3p1-70b-instruct"):
    railguards.META_MODEL = model
    
    related_regulations = await get_related_regulations(qa)
    my_system_prompt = (
        f"{system_prompt}\n các điều sau có thể có liên quan: {related_regulations}"
    )
    print(my_system_prompt)
    return await railguards.execute_chat_with_guardrail(
        qa, "đơn khiếu nại", system_prompt=my_system_prompt
    )

In [94]:
async def get_related_regulations(qa):
    
    prompt = f"""trong các luật sau, luật nào có liên quan tới việc {qa}. Trả lời dưới dạng một json list. 
    {all_laws}
    """
    applicable_laws = await railguards.get_chat_response(prompt, "")
    applicable_laws = json.loads(
        selected_laws.replace("```json", "").replace("```", "")
    )
    retrieved_nodes = []

    for applicable_law in applicable_laws:
        filters = MetadataFilters(
            filters=[
                MetadataFilter(
                    key="law",
                    operator=FilterOperator.EQ,
                    value=applicable_law,  # all_laws.split("\n")
                )
            ]
        )
        retriever = VectorIndexRetriever(
            index=index,
            filters=filters,
            # You can adjust the number of results retrieved
            similarity_top_k=2,
        )
        nodes_re = await retriever.aretrieve(qa)
        retrieved_nodes.extend(nodes_re)
    related_regulations = ""
    for node in retrieved_nodes:
        related_regulations += f"{node.text}\n({node.metadata['citation']})\n-------\n"
    return related_regulations


In [ ]:
print(await get_llm_answer(f))